In [36]:
import pandas as pd 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

c:\Users\Noah7\code\Github\AI-Noah-Eliasson-OPA23\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [29]:
movies="C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/movies.csv"
ratings="C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/ratings.csv"

df_movies=pd.read_csv(movies, usecols=["movieId", "title"], dtype={"movieId": "int32", "title": "str"})
df_ratings=pd.read_csv(ratings, usecols=["userId", "movieId", "rating"], dtype={"userId": "int32", "movieId": "int32", "rating": "float32"})

In [44]:
# spare matrix 
#        Users
#        [4,4,5]
# movies [4,3,2]
#        [3,1,3]

movies_users= df_ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_movies_users= csr_matrix(movies_users.values)

mat_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [33]:
# Eculidean distance 
# Manhattan distance
# Minkowski distance
# Cosine similarity  
model_KNN= NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=20)

In [34]:
model_KNN.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [43]:
# Recommender(movie_name) => List of Movies recommended 

def recommender(movie_name, data, model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies["title"])[2]
    print("Movies Selected: ", df_movies["title"][idx], "Index: ", idx)
    print("Searching for recommendations.....")
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i!=idx))
        

recommender("iron man 2", mat_movies_users, model_KNN, 20)

Movies Selected:  Iron Man 2 (2010) Index:  7324
Searching for recommendations.....
9656                     The Greatest Showman (2017)
8590                           Jurassic World (2015)
7324                                             NaN
8946    Kung Fu Panda: Secrets of the Masters (2011)
9605           The Nut Job 2: Nutty by Nature (2017)
7936                          Treasure Island (2012)
9447                       Microwave Massacre (1983)
9665                       Porky in Wackyland (1938)
8865                              The Lobster (2015)
9600                            Der Herr Karl (1961)
9300                             Kill Command (2016)
7028                              Powder Blue (2009)
6600                The Jane Austen Book Club (2007)
9479                           American Fable (2017)
9629                             Paddington 2 (2017)
7569                                Insidious (2010)
9354                        Vigilante Diaries (2016)
1707           